In [1]:
source("../00_model-fitting/wwa_model-fitting.R")

## **Linear trend in log10_cumDSR**

### **Prep data**

In [2]:
# load data
gmst <- read.fwf("/home/clair/Data/gmst.dat", comment.char = "#", widths = c(5,15), col.names = c("year", "gmst"))
cdsr <- read.csv("data/era5_cdsr.csv")

# take base10 log of DSR as in original study
log10_cdsr <- sapply(colnames(cdsr)[-1], function(cnm) log10(cdsr[,cnm]))
colnames(log10_cdsr) <- paste0("log10_", colnames(log10_cdsr))       
log10_cdsr <- data.frame("year" = cdsr$year, log10_cdsr)
                     
# merge into single dataframe
df <- merge(gmst, log10_cdsr)

### **Check distribution of log10_cumDSR**

In [3]:
prep_window(c(4,4))
x <- seq(0.6,3,0.01)

jpeg("fig/expl_era5_cdsr-fit.jpg", width = 480 * 3, height = 480 * 2); par(mfrow = c(4,4), lwd = 2, cex = 1.2, mar = c(3,3,2,1)); {
    invisible(sapply(colnames(df)[-(1:2)], function(cnm) { 
        ts <- df[,cnm]
        hist(ts, prob = T, xlim = c(0.8,3), ylim = c(0,3), main = cnm, xlab = "", ylab = "")
        lines(density(ts))
        lines(x, dnorm(x, mean(ts), sd(ts)), col = "blue")
    }))
}; dev.off()

png 
  2

### **Fit linear trend model**

In [4]:
models <- sapply(colnames(df)[-(1:2)], function(cnm) fit_ns("norm", "shift", df, cnm, "gmst", lower = F), simplify = F)
gmst_2023 <- gmst[gmst$year == 2023, "gmst"]
gmst_hist <- gmst_2023 - 1.2

In [5]:
res <- sapply(models, function(mdl) mdl_ests(mdl, cov = gmst_2023, cov_cf = gmst_hist, ev = mdl$ev))

In [6]:
write.csv(res, file = "res/cdsr_lineartrend-ests.csv")

In [7]:
prep_window(c(4,4))

jpeg("fig/expl_era5_cdsr-trend.jpg", width = 480 * 3, height = 480 * 2); par(mfrow = c(4,4), lwd = 2, cex = 1.2, mar = c(3,3,2,1)); {
    invisible(lapply(models, function(mdl) { 
        plot_trend(mdl, legend_pos = NA, main = mdl$varnm)
        lines(df$year, filter(mdl$x, rep(1/15,15), sides = 2), col = "forestgreen", lwd = 2)
    }))
}; dev.off()

png 
  2

In [8]:
prep_window(c(4,4))

jpeg("fig/expl_era5_cdsr-gmsttrend.jpg", width = 480 * 3, height = 480 * 2); par(mfrow = c(4,4), lwd = 2, cex = 1.2, mar = c(3,3,2,1)); {
    invisible(lapply(models, function(mdl) { 
        plot_gmsttrend(mdl, legend_pos = NA, main = mdl$varnm, cov = gmst_2023, cov_cf = gmst_hist)
        lines(df$gmst[order(df$gmst)], filter(mdl$x[order(df$gmst)], rep(1/15,15), sides = 2), col = "forestgreen", lwd = 2)
    }))
}; dev.off()

png 
  2

In [9]:
prep_window(c(4,4))

jpeg("fig/expl_era5_cdsr-rlplots.jpg", width = 480 * 3, height = 480 * 2); par(mfrow = c(4,4), lwd = 2, cex = 1.2, mar = c(3,3,2,1)); {
    invisible(lapply(models, function(mdl) { 
        plot_returnlevels(mdl, legend_pos = NA, main = mdl$varnm, cov = gmst_2023, cov_cf = gmst_hist)
    }))
}; dev.off()

png 
  2

---
## **Sensitivity to start of time series (ERA5)**

In [10]:
# load data
gmst <- read.fwf("/home/clair/Data/gmst.dat", comment.char = "#", widths = c(5,15), col.names = c("year", "gmst"))
cdsr <- read.csv("data/era5_cdsr.csv")

# take base10 log of DSR as in original study
log10_cdsr <- sapply(colnames(cdsr)[-1], function(cnm) log10(cdsr[,cnm]))
colnames(log10_cdsr) <- paste0("log10_", colnames(log10_cdsr))       
log10_cdsr <- data.frame("year" = cdsr$year, log10_cdsr)
                     
# merge into single dataframe
df <- merge(gmst, log10_cdsr)
                     
# fit all the models, also excluding years prior to 1950/1960
models_1940 <- sapply(colnames(df)[-(1:2)], function(cnm) fit_ns("norm", "shift", df[df$year >= 1940,], cnm, "gmst", lower = F), simplify = F)
models_1950 <- sapply(colnames(df)[-(1:2)], function(cnm) fit_ns("norm", "shift", df[df$year >= 1950,], cnm, "gmst", lower = F), simplify = F)
models_1960 <- sapply(colnames(df)[-(1:2)], function(cnm) fit_ns("norm", "shift", df[df$year >= 1960,], cnm, "gmst", lower = F), simplify = F)

gmst_2023 <- gmst[gmst$year == 2023, "gmst"]
gmst_hist <- gmst_2023 - 1.2

In [11]:
prep_window(c(4,4))

jpeg("fig/expl-sa_era5_cdsr-trend_start-sensitivity.jpg", width = 480 * 3, height = 480 * 2); par(mfrow = c(4,4), lwd = 2, cex = 1.2, mar = c(3,3,2,1)); {
    invisible(sapply(1:16, function(i) {    
        plot(df$year, df[,-(1:2)][,i], type = "S", main = models_1940[[i]]$varnm)

        lines(models_1940[[i]]$data$year, ns_pars(models_1940[[i]])$loc)
        lines(models_1950[[i]]$data$year, ns_pars(models_1950[[i]])$loc, col = "darkblue")
        lines(models_1960[[i]]$data$year, ns_pars(models_1960[[i]])$loc, col = "darkred")
        
        lines(df$year, filter(models_1940[[i]]$x, rep(1/15,15), sides = 2), col = "forestgreen", lwd = 2)

    }))
}; dev.off()

png 
  2

In [12]:
prep_window(c(4,4))

jpeg("fig/expl-sa_era5_cdsr-gmsttrend_start-sensitivity.jpg", width = 480 * 3, height = 480 * 2); par(mfrow = c(4,4), lwd = 2, cex = 1.2, pch = 20, mar = c(3,3,2,1)); {
    invisible(sapply(1:16, function(i) {    
        plot(df$gmst, df[,-(1:2)][,i], main = models_1940[[i]]$varnm, ylab = models_1940[[i]]$varnm)

        lines(models_1940[[i]]$data$gmst, ns_pars(models_1940[[i]])$loc)
        lines(models_1950[[i]]$data$gmst, ns_pars(models_1950[[i]])$loc, col = "darkblue")
        lines(models_1960[[i]]$data$gmst, ns_pars(models_1960[[i]])$loc, col = "darkred")
        
        lines(df$gmst, eff_return_level(6, models_1940[[i]], fixed_cov = df$gmst))
        lines(df$gmst, eff_return_level(6, models_1950[[i]], fixed_cov = df$gmst), col = "darkblue")
        lines(df$gmst, eff_return_level(6, models_1960[[i]], fixed_cov = df$gmst), col = "darkred")
        
        lines(df$gmst[order(df$gmst)], filter(models_1940[[i]]$x[order(df$gmst)], rep(1/15,15), sides = 2), col = "forestgreen", lwd = 2)
        points(df[df$year == 2023, c("gmst", models_1940[[i]]$varnm)], pch = 0, col = "magenta", lwd = 2)

    }))
}; dev.off()

png 
  2